# Why saving models?

Save a model and continue training at a later time. It is a checkpoint of the model.

In [13]:
# Imports
import torch
import torch.nn as nn  # all NN modules
import torch.nn.functional as F  # RElu, Tanh, ...
import torch.optim as optim  # SGD, Adam, ...
import torchvision.datasets as datasets  # MNIST dataset
import torchvision.transforms as transforms  # Tranformations on dataset
from torch.utils.data import DataLoader  # Dataset management and minibatches


In [14]:
# Create a simple CNN
class CNN(nn.Module):
    def __init__(self, in_channel=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channel,
            out_channels=8,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )  # same convolution
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )  # same convolution
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x


In [15]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    """
    state is a dictionary
    """
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])


In [16]:
# Set device
if torch.cuda.is_available():
    device = "gpu"
elif torch.has_mps:
    device = "mps"
else:
    device = "cpu"
device = torch.device(device)
print(device)


mps


In [17]:
# Hyper parameters
in_channel = 1
num_classes = 10
learning_rate = 1e-4
batch_size = 1024
num_epochs = 10
load_model = False


In [18]:
# Load data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [19]:
# Initialize network
model = CNN(in_channel=in_channel, num_classes=num_classes).to(device)


In [20]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [21]:
if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)


In [22]:
# This cell took 20 seconds for 10 epochs on MacBook Pro (16-inch, 2021) Apple M1 Pro 16 GB

# Train network
for epoch in range(num_epochs):  # epoch
    losses = []

    if epoch % 3 == 0:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

    for batch_idx, (data, targets) in enumerate(train_loader):  # batch
        # Get data to CUDA if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or Adam step
        optimizer.step()  # update the weights

    print(f"Loss at epoch {epoch} was {loss.item()}")


=> Saving checkpoint
Loss at epoch 0 was 2.228992223739624
Loss at epoch 1 was 1.9985829591751099
Loss at epoch 2 was 1.5614322423934937
=> Saving checkpoint
Loss at epoch 3 was 1.1266452074050903
Loss at epoch 4 was 0.8270728588104248
Loss at epoch 5 was 0.6476271152496338
=> Saving checkpoint
Loss at epoch 6 was 0.5782060623168945
Loss at epoch 7 was 0.4498629868030548
Loss at epoch 8 was 0.3800198435783386
=> Saving checkpoint
Loss at epoch 9 was 0.39147403836250305


In [23]:
# Check accuracy on training and test to see how good our moodel is
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            # 64x10
            _, predictions = scores.max(dim=1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print(
            f"Got {num_correct} / {num_samples} with accuracy "
            f"{float(num_correct) / float(num_samples) * 100:.2f}"
        )

    model.train()
    return float(num_correct) / float(num_samples)


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


Checking accuracy on training data
Got 53530 / 60000 with accuracy 89.22
Checking accuracy on test data
Got 8983 / 10000 with accuracy 89.83


0.8983